In [29]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 18: Duet ---</h2><p>You discover a tablet containing some strange assembly code labeled simply "<a href="https://en.wikipedia.org/wiki/Duet">Duet</a>". Rather than bother the sound card with it, you decide to run the code yourself. Unfortunately, you don't see any documentation, so you're left to figure out what the instructions mean on your own.</p>
<p>It seems like the assembly is meant to operate on a set of <em>registers</em> that are each named with a single letter and that can each hold a single <a href="https://en.wikipedia.org/wiki/Integer">integer</a>. You suppose each register should start with a value of <code>0</code>.</p>
<p>There aren't that many instructions, so it shouldn't be hard to figure out what they do.  Here's what you determine:</p>
<ul>
<li><code>snd X</code> <em><span title="I don't recommend actually trying this.">plays a sound</span></em> with a frequency equal to the value of <code>X</code>.</li>
<li><code>set X Y</code> <em>sets</em> register <code>X</code> to the value of <code>Y</code>.</li>
<li><code>add X Y</code> <em>increases</em> register <code>X</code> by the value of <code>Y</code>.</li>
<li><code>mul X Y</code> sets register <code>X</code> to the result of <em>multiplying</em> the value contained in register <code>X</code> by the value of <code>Y</code>.</li>
<li><code>mod X Y</code> sets register <code>X</code> to the <em>remainder</em> of dividing the value contained in register <code>X</code> by the value of <code>Y</code> (that is, it sets <code>X</code> to the result of <code>X</code> <a href="https://en.wikipedia.org/wiki/Modulo_operation">modulo</a> <code>Y</code>).</li>
<li><code>rcv X</code> <em>recovers</em> the frequency of the last sound played, but only when the value of <code>X</code> is not zero. (If it is zero, the command does nothing.)</li>
<li><code>jgz X Y</code> <em>jumps</em> with an offset of the value of <code>Y</code>, but only if the value of <code>X</code> is <em>greater than zero</em>. (An offset of <code>2</code> skips the next instruction, an offset of <code>-1</code> jumps to the previous instruction, and so on.)</li>
</ul>
<p>Many of the instructions can take either a register (a single letter) or a number. The value of a register is the integer it contains; the value of a number is that number.</p>
<p>After each <em>jump</em> instruction, the program continues with the instruction to which the <em>jump</em> jumped. After any other instruction, the program continues with the next instruction. Continuing (or jumping) off either end of the program terminates it.</p>
<p>For example:</p>
<pre><code>set a 1
add a 2
mul a a
mod a 5
snd a
set a 0
rcv a
jgz a -1
set a 1
jgz a -2
</code></pre>
<ul>
<li>The first four instructions set <code>a</code> to <code>1</code>, add <code>2</code> to it, square it, and then set it to itself modulo <code>5</code>, resulting in a value of <code>4</code>.</li>
<li>Then, a sound with frequency <code>4</code> (the value of <code>a</code>) is played.</li>
<li>After that, <code>a</code> is set to <code>0</code>, causing the subsequent <code>rcv</code> and <code>jgz</code> instructions to both be skipped (<code>rcv</code> because <code>a</code> is <code>0</code>, and <code>jgz</code> because <code>a</code> is not greater than <code>0</code>).</li>
<li>Finally, <code>a</code> is set to <code>1</code>, causing the next <code>jgz</code> instruction to activate, jumping back two instructions to another jump, which jumps again to the <code>rcv</code>, which ultimately triggers the <em>recover</em> operation.</li>
</ul>
<p>At the time the <em>recover</em> operation is executed, the frequency of the last sound played is <code>4</code>.</p>
<p><em>What is the value of the recovered frequency</em> (the value of the most recently played sound) the <em>first</em> time a <code>rcv</code> instruction is executed with a non-zero value?</p>
</article>


In [30]:
from re import compile, match

from more_itertools import first


example = """
set a 1
add a 2
mul a a
mod a 5
snd a
set a 0
rcv a
jgz a -1
set a 1
jgz a -2
"""


def duet(program: str) -> int:
    send = None
    registers = defaultdict(int)

    regex = compile(r"^[\-+]?\d+$")

    instuctions = [
        [int(c) if match(regex, c) else c for c in line.split(" ")]
        for line in program.strip().splitlines()
    ]

    counter = 0
    while 0 <= counter < len(instuctions):
        match instuctions[counter]:
            case ["snd", str(x)]:
                send = registers[x]
            case ["snd", int(x)]:
                send.append(x)
            case ["set", str(x), str(y)]:
                registers[x] = registers[y]
            case ["set", str(x), int(y)]:
                registers[x] = y
            case ["add", str(x), str(y)]:
                registers[x] += registers[y]
            case ["add", str(x), int(y)]:
                registers[x] += y
            case ["mul", str(x), str(y)]:
                registers[x] *= registers[y]
            case ["mul", str(x), int(y)]:
                registers[x] *= y
            case ["mod", str(x), str(y)]:
                registers[x] %= registers[y]
            case ["mod", str(x), int(y)]:
                registers[x] %= y
            case ["rcv", str(x)]:
                if registers[x] > 0:
                    yield send
            case ["jgz", str(x), str(y)]:
                if registers[x] > 0:
                    counter += registers[y] - 1
            case ["jgz", str(x), int(y)]:
                if registers[x] > 0:
                    counter += y - 1
            case ["jgz", int(x), str(y)]:
                if x > 0:
                    counter += registers[y] - 1
            case ["jgz", int(x), int(y)]:
                if x > 0:
                    counter += y - 1
            case _:
                raise ValueError(f"Unknown instruction {instuctions[counter]}.")

        counter += 1


print(f"Example: {first(duet(example))} should be 4")

Example: 4 should be 4


In [31]:
with open("../input/day18.txt") as f:
    puzzle = f.read()


print(f"Part I: {first(duet(puzzle))}")

Part I: 4601


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>4601</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>As you congratulate yourself for a job well done, you notice that the documentation has been on the back of the tablet this entire time. While you actually got most of the instructions correct, there are a few key differences. This assembly code isn't about sound at all - it's meant to be run <em>twice at the same time</em>.</p>
<p>Each running copy of the program has its own set of registers and follows the code independently - in fact, the programs don't even necessarily run at the same speed. To coordinate, they use the <em>send</em> (<code>snd</code>) and <em>receive</em> (<code>rcv</code>) instructions:</p>
<ul>
<li><code>snd X</code> <em>sends</em> the value of <code>X</code> to the other program. These values wait in a queue until that program is ready to receive them. Each program has its own message queue, so a program can never receive a message it sent.</li>
<li><code>rcv X</code> <em>receives</em> the next value and stores it in register <code>X</code>. If no values are in the queue, the program <em>waits for a value to be sent to it</em>. Programs do not continue to the next instruction until they have received a value. Values are received in the order they are sent.</li>
</ul>
<p>Each program also has its own <em>program ID</em> (one <code>0</code> and the other <code>1</code>); the register <code>p</code> should begin with this value.</p>
<p>For example:</p>
<pre><code>snd 1
snd 2
snd p
rcv a
rcv b
rcv c
rcv d
</code></pre>
<p>Both programs begin by sending three values to the other.  Program <code>0</code> sends <code>1, 2, 0</code>; program <code>1</code> sends <code>1, 2, 1</code>. Then, each program receives a value (both <code>1</code>) and stores it in <code>a</code>, receives another value (both <code>2</code>) and stores it in <code>b</code>, and then each receives the program ID of the other program (program <code>0</code> receives <code>1</code>; program <code>1</code> receives <code>0</code>) and stores it in <code>c</code>. Each program now sees a different value in its own copy of register <code>c</code>.</p>
<p>Finally, both programs try to <code>rcv</code> a <em>fourth</em> time, but no data is waiting for either of them, and they reach a <em>deadlock</em>.  When this happens, both programs terminate.</p>
<p>It should be noted that it would be equally valid for the programs to run at different speeds; for example, program <code>0</code> might have sent all three values and then stopped at the first <code>rcv</code> before program <code>1</code> executed even its first instruction.</p>
<p>Once both of your programs have terminated (regardless of what caused them to do so), <em>how many times did program <code>1</code> send a value</em>?</p>
</article>

</main>


In [32]:
from collections import deque


def duet_II(program: str) -> int:
    regex = compile(r"^[\-+]?\d+$")

    instuctions = [
        [int(c) if match(regex, c) else c for c in line.split(" ")]
        for line in program.strip().splitlines()
    ]

    counters = [0, 0]
    did_send = [0, 0]
    blocked = [False, False]
    registers = [defaultdict(int), defaultdict(int)]
    registers[0]["p"] = 0
    registers[1]["p"] = 1
    queues_sending = [deque([]), deque([])]

    pc = 0

    while not all(blocked):
        while 0 <= counters[pc] < len(instuctions):
            match instuctions[counters[pc]]:
                case ["snd", str(x)]:
                    did_send[pc] += 1
                    queues_sending[pc].append(registers[pc][x])
                case ["snd", int(x)]:
                    did_send[pc] += 1
                    queues_sending[pc].append(x)
                case ["set", str(x), str(y)]:
                    registers[pc][x] = registers[pc][y]
                case ["set", str(x), int(y)]:
                    registers[pc][x] = y
                case ["add", str(x), str(y)]:
                    registers[pc][x] += registers[pc][y]
                case ["add", str(x), int(y)]:
                    registers[pc][x] += y
                case ["mul", str(x), str(y)]:
                    registers[pc][x] *= registers[pc][y]
                case ["mul", str(x), int(y)]:
                    registers[pc][x] *= y
                case ["mod", str(x), str(y)]:
                    registers[pc][x] %= registers[pc][y]
                case ["mod", str(x), int(y)]:
                    registers[pc][x] %= y
                case ["rcv", str(x)]:
                    if queues_sending[1 - pc]:
                        registers[pc][x] = queues_sending[pc ^ 1].popleft()
                    else:
                        blocked[pc] = True
                        break
                case ["jgz", str(x), str(y)]:
                    if registers[pc][x] > 0:
                        counters[pc] += registers[pc][y] - 1
                case ["jgz", str(x), int(y)]:
                    if registers[pc][x] > 0:
                        counters[pc] += y - 1
                case ["jgz", int(x), str(y)]:
                    if x > 0:
                        counters[pc] += registers[pc][y] - 1
                case ["jgz", int(x), int(y)]:
                    if x > 0:
                        counters[pc] += y - 1
                case _:
                    raise ValueError(
                        f"Unknown instruction {instuctions[counters[pc]]}."
                    )

            counters[pc] += 1

        if blocked[pc ^ 1] and queues_sending[pc]:
            blocked[pc ^ 1] = False

        pc ^= 1

    return did_send[1]


example = """
snd 1
snd 2
snd p
rcv a
rcv b
rcv c
rcv d
"""

print(f"Example: {duet_II(example)} should be 3")

Example: 3 should be 3


In [33]:
print(f"Part II: {duet_II(puzzle)}")

Part II: 6858


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>6858</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
